In [1]:
# Parameters
run_mode = "latest"
run_date = None
stage_path = "/opt/airflow/data-layer/raw/stage"
raw_path = "/opt/airflow/data-layer/raw"
silver_path = "/opt/airflow/data-layer/silver"
postgres_conn_id = "dw"


# etl_raw_to_silver
---
Este notebook executa o processo `ETL` que transfere os dados da camada **Raw** para a **Silver**, englobando transformações, limpeza, padronização, movimentação de arquivos e materialização dos resultados tanto em arquivos `parquet` na camada **Silver** quanto em tabelas correspondentes no *PostgreSQL*. Além disso, ele realiza a carga parcial dos dados da **Raw** para inicializar a pipeline com *dbt*.


In [2]:
# Parameters

run_mode = "latest"
run_date = None

stage_path = "/opt/airflow/data-layer/raw/stage"
raw_path = "/opt/airflow/data-layer/raw"
silver_path = "/opt/airflow/data-layer/silver"

postgres_conn_id = "AIRFLOW_VAR_POSTGRES_CONN_ID"


In [3]:
import os
from datetime import datetime
import shutil
from pathlib import Path

from pyspark import StorageLevel
from pyspark.sql import DataFrame, functions as F, Window, SparkSession
from pyspark.sql.column import Column
from pyspark.sql.types import DoubleType, StringType

from transformer.utils.file_io import check_files_in_folder, delete_files, find_partition, move_files
from transformer.utils.helpers import to_date_from_ymd
from transformer.utils.logger import get_logger
from transformer.utils.spark_helpers import get_spark_session, load_to_postgres
from transformer.utils.postgre_helpers import assert_table_rowcount
from transformer.utils.quality_gates_raw import run_quality_gates_raw
from transformer.utils.quality_gates_silver_aggregated import run_quality_gates_silver_aggregated
from transformer.utils.quality_gates_silver_base import run_quality_gates_silver_base
from transformer.utils.quality_gates_silver_flights import run_quality_gates_silver_flights


# Raw

## Job 0: kaggle_download_and_prepare

Este job executa o download, a descompactação e a preparação dos arquivos na pasta **Stage**. Sua execução permanece desativada porque depende de configurações manuais de acesso à API do Kaggle. Reative as células e informe suas credenciais caso deseje executá-la.


In [4]:
%%script false --no-raise-error # Comentar essa linha se quiser rodar a célula.
import numpy as np
import pandas as pd

log = get_logger("kaggle_and_prepare")

NUM_CHUNKS=10


### Definindo funções de download e de divisão

In [5]:
%%script false --no-raise-error # Comentar essa linha se quiser rodar a célula.
def download_kaggle_dataset(dataset_name: str, dest_path: Path) -> None:
    """
    Faz o download e a extração de um dataset do Kaggle, caso ainda não exista localmente.

    Args:
        dataset_name (str): Nome do dataset no Kaggle (ex: 'usdot/flight-delays').
        dest_path (Path): Caminho destino para os arquivos extraídos.

    Raises:
        Exception: Para erros de autenticação ou rede.
    """
    try:
        existing_files = os.listdir(dest_path)
        has_csv_files = any(file.endswith(".csv") for file in existing_files)

        if has_csv_files:
            log.info(f"[Kaggle] Arquivos já existem em '{dest_path}'. Pulando download.")
            return

        log.info(f"[Kaggle] Nenhum arquivo CSV encontrado. Baixando o dataset '{dataset_name}'.")
        kaggle.api.dataset_download_files(dataset_name, path=dest_path, unzip=True)
        log.info("[Kaggle] Download e extração concluídos com sucesso.")
    except Exception as e:
        log.exception(f"[Kaggle] Ocorreu um erro! Verifique se a API do Kaggle está configurada: {e}.")
        log.exception(f"[Kaggle] Erro: {e}.")
        raise

def split_main_file(source_file: Path, num_chunks: int = NUM_CHUNKS) -> None:
    """
    Divide o arquivo flights.csv em múltiplos chunks de tamanho aproximadamente igual.

    Args:
        source_file (Path): Caminho do arquivo csv principal.
        num_chunks (int): Número de partes a gerar.

    Raises:
        FileNotFoundError: Se o arquivo não existir.
        ValueError: Se o arquivo estiver vazio.
        Exception: Para erros inesperados durante a divisão.
    """
    if not source_file.exists():
        raise FileNotFoundError(f"[Landing] Arquivo não encontrado: '{source_file}'.")

    try:
        df = pd.read_csv(source_file)
        if df.empty:
            raise ValueError(f"O arquivo '{source_file}' está vazio.")

        log.info(f"[Split] Dividindo '{source_file.name}' em {num_chunks} partes aproximadamente iguais.")

        chunks = np.array_split(df, num_chunks)

        for i, chunk_df in enumerate(chunks, start=1):
            chunk_name = f"{source_file.stem}_part_{i:02d}{source_file.suffix}"
            chunk_path = source_file.parent / chunk_name
            chunk_df.to_csv(chunk_path, index=False)
            log.info(f"[Split] Chunk salvo: '{chunk_name}', {len(chunk_df)} linhas.")

        log.info("[Split] Divisão do arquivo concluída com sucesso.")

    except Exception as e:
        log.error(f"[Split] Erro ao dividir o arquivo: {e}.")
        raise


### Runner para o job `kaggle_download_and_prepare`

In [6]:
%%script false --no-raise-error # Comentar essa linha se quiser rodar a célula.
try:
    # Download
    download_kaggle_dataset(DATASET_NAME, stage_path)

    # Divisão
    flights_file = Path(stage_path) / "flights.csv"
    if flights_file.exists():
        split_main_file(flights_file, num_chunks=NUM_CHUNKS)
    else:
        log.warning(f"[Kaggle] Arquivo 'flights.csv' não encontrado. Verifique a pasta.")

except Exception as e:
    log.error(f"[Kaggle] Falha no processo de preparação: {e}")


## Job 1: unify_flight_chunks

Este job realiza a unificação dos arquivos `flights_part_*.csv` presentes na pasta **Stage**, consolidando-os em um único arquivo Parquet `flights.parquet`.

In [7]:
log = get_logger("unify_chunks")

spark = get_spark_session("UnifyFlightChunks")
log.info("[UnifyChunks] SparkSession iniciada.")


/usr/local/lib/python3.12/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found


:: loading settings :: url = jar:file:/usr/local/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /root/.ivy2/cache
The jars for the packages stored in: /root/.ivy2/jars
org.postgresql#postgresql added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-4876e2b5-1729-40ee-8a77-e086cd52fa50;1.0
	confs: [default]


	found org.postgresql#postgresql;42.7.3 in central


	found org.checkerframework#checker-qual;3.42.0 in central


...

	[SUCCESSFUL ] org.postgresql#postgresql;42.7.3!postgresql.jar (621ms)
downloading https://repo1.maven.org/maven2/org/checkerframework/checker-qual/3.42.0/checker-qual-3.42.0.jar ...
	[SUCCESSFUL ] org.checkerframework#checker-qual;3.42.0!checker-qual.jar (173ms)
:: resolution report :: resolve 1399ms :: artifacts dl 802ms
	:: modules in use:
	org.checkerframework#checker-qual;3.42.0 from central in [default]
	org.postgresql#postgresql;42.7.3 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   2   |   2   |   2   |   0   ||   2   |   2   |
	---------------------------------------------------------------------
:: retrieving :: org.apache.spark#spark-submit-parent-4876e2b5-1729-40ee-8a77-e086cd52fa50
	confs: [d

25/11/27 02:46:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


2025-11-27 02:46:44 [INFO] spark_helpers: [INFO] SparkSession criada: 'UnifyFlightChunks' (master=local[*]).


2025-11-27 02:46:44 [INFO] unify_chunks: [UnifyChunks] SparkSession iniciada.


### Definindo função de unificação

In [8]:
def reassemble_chunks(spark: SparkSession, chunk_files: list[str], header: bool = True) -> DataFrame:
    """
    Lê múltiplos arquivos de chunk (flights_part_*.csv) e os unifica em um único DataFrame Spark.

    Args:
        spark (SparkSession): Sessão Spark ativa.
        chunk_files (list[str]): Lista de caminhos completos dos arquivos csv a serem unificados.
        header (bool, opcional): Define se os arquivos csv possuem cabeçalho. Padrão: True.

    Returns:
        DataFrame: DataFrame Spark consolidado com todos os chunks.
    """
    if not chunk_files:
        raise ValueError("[UnifyChunks][Error] Nenhum arquivo de chunk fornecido para unificação.")

    log.info(f"[UnifyChunks] Lendo e concatenando {len(chunk_files)} arquivo(s) de chunk.")

    try:
        df = (
            spark.read
            .option("header", header)
            .option("inferSchema", False)
            .csv(chunk_files)
        )
        
        log.info("[UnifyChunks] Unificação dos chunks concluída com sucesso.")
        
        return df

    except Exception as e:
        log.error(f"[UnifyChunks][Error] Falha ao ler os arquivos csv: {e}.")
        
        raise


### Runner para o job `unify_flight_chunks`

In [9]:
try:
    log.info("[UnifyChunks] Iniciando job de unificação de chunks.")

    # Localiza arquivos csv na stage
    csv_files = check_files_in_folder(stage_path, "*.csv")
    chunk_files = [f for f in csv_files if "flights_part" in f]

    if not chunk_files:
        raise FileNotFoundError(f"[UnifyChunks][Error] Nenhum arquivo de chunk encontrado em {stage_path}.")

    # Unifica os chunks
    df_unified = reassemble_chunks(spark, chunk_files)

    # Executa quality gates
    required_columns = [
        'YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER', 'TAIL_NUMBER', 
        'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT', 'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 
        'DEPARTURE_DELAY', 'TAXI_OUT', 'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 
        'AIR_TIME', 'DISTANCE', 'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME', 
        'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON', 'AIR_SYSTEM_DELAY', 
        'SECURITY_DELAY', 'AIRLINE_DELAY', 'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'
    ]
    run_quality_gates_raw(df_unified, "raw_flights", required_columns)

    # Salva o arquivo unificado
    stage_output = f"{stage_path}/flights.parquet"
    df_unified.write.mode("overwrite").option("compression", "snappy").parquet(stage_output)
    
    log.info(f"[UnifyChunks] Arquivo unificado salvo em: {stage_output}.")

except Exception as e:
    log.exception(f"[UnifyChunks][Error] Falha durante execução: {e}")
    raise

finally:
    log.info("[UnifyChunks] Job de unificação de chunks encerrado.")


2025-11-27 02:46:44 [INFO] unify_chunks: [UnifyChunks] Iniciando job de unificação de chunks.


2025-11-27 02:46:44 [INFO] file_io: [INFO] Encontrados 12 arquivo(s).


2025-11-27 02:46:44 [INFO] unify_chunks: [UnifyChunks] Lendo e concatenando 10 arquivo(s) de chunk.


2025-11-27 02:46:49 [INFO] unify_chunks: [UnifyChunks] Unificação dos chunks concluída com sucesso.


2025-11-27 02:46:49 [INFO] quality_gates_raw: [Quality][Raw] Iniciando validações do dataset 'raw_flights'.


2025-11-27 02:46:51 [INFO] quality_gates_raw: [Quality][Raw]       _check_row_count_not_empty: raw_flights OK.


2025-11-27 02:46:51 [INFO] quality_gates_raw: [Quality][Raw]       _check_schema_columns: raw_flights OK.


2025-11-27 02:46:51 [INFO] quality_gates_raw: [Quality][Raw] Todas as validações para 'raw_flights' concluídas com sucesso.


2025-11-27 02:47:25 [INFO] unify_chunks: [UnifyChunks] Arquivo unificado salvo em: /opt/airflow/data-layer/raw/stage/flights.parquet.


2025-11-27 02:47:25 [INFO] unify_chunks: [UnifyChunks] Job de unificação de chunks encerrado.


In [10]:
# Encerra a sessão Spark para o job
spark.stop()
log.info("[UnifyChunks] Sessão Spark finalizada.")


2025-11-27 02:47:26 [INFO] unify_chunks: [UnifyChunks] Sessão Spark finalizada.


## Job 2: convert_csv_to_parquet

Este job realiza a conversão dos arquivos csv presentes na pasta **Stage** (exceto os chunks `flights_part_*`) para o formato `parquet`, mantendo o schema original dos dados.


In [11]:
log = get_logger("convert_csv")

spark = get_spark_session("ConvertCsvToParquet")
log.info("[ConvertCSV] SparkSession iniciada.")


2025-11-27 02:47:26 [INFO] spark_helpers: [INFO] SparkSession criada: 'ConvertCsvToParquet' (master=local[*]).


2025-11-27 02:47:26 [INFO] convert_csv: [ConvertCSV] SparkSession iniciada.


### Definindo função de conversão

In [12]:
def convert_csv_to_parquet(spark: SparkSession, csv_files: list[str], stage_path: str) -> None:
    """
    Converte arquivos csv em formato parquet.

    Args:
        spark (SparkSession): Sessão Spark ativa.
        csv_files (list[str]): Lista de caminhos dos arquivos csv a converter.
        stage_path (str): Caminho base da camada Stage.
    """
    if not csv_files:
        raise ValueError("[ConvertCSV][Error] Nenhum arquivo csv fornecido para conversão.")

    for csv in csv_files:
        try:
            # Resolve caminhho do arquivo
            base_name = os.path.basename(csv).replace(".csv", ".parquet")

            log.info(f"[ConvertCSV] Lendo arquivo csv: {csv}.")

            df = (
                spark.read
                .option("header", True)
                .option("inferSchema", False)
                .csv(csv)
            )

            # Executa quality gates
            if df.columns[1] == 'AIRLINE':
                required_columns = ['IATA_CODE', 'AIRLINE']
            else: 
                required_columns = ['IATA_CODE', 'AIRPORT', 'CITY', 'STATE', 'COUNTRY', 'LATITUDE', 'LONGITUDE']
            
            run_quality_gates_raw(df, base_name, required_columns)

            # Converte em parquet
            parquet_path = f"{stage_path}/{base_name}"
            df.write.mode("overwrite").option("compression", "snappy").parquet(parquet_path)
            
            log.info(f"[ConvertCSV] Arquivo convertido: {parquet_path}.")

        except Exception as e:
            log.error(f"[ConvertCSV][ERROR] Falha ao converter {csv}: {e}.")
            raise IOError(f"Erro ao processar {csv}: {e}.") from e


### Runner para o job `convert_csv_to_parquet`

In [13]:
try:
    log.info("[ConvertCSV] Iniciando job de conversão de csv para parquet.")

    csv_files = check_files_in_folder(stage_path, "*.csv")
    target_files = [f for f in csv_files if "flights_part" not in f]

    if not target_files:
        raise FileNotFoundError(f"[ConvertCSV][Error] Nenhum arquivo csv com o padrão encontrado em {stage_path}.")

    convert_csv_to_parquet(spark, target_files, stage_path)

    log.info(f"[ConvertCSV] Conversão concluída. {len(target_files)} arquivo(s) processado(s).")

except Exception as e:
    log.exception(f"[ConvertCSV][Error] Falha durante execução: {e}.")
    raise

finally:
    log.info("[ConvertCSV] Job de conversão encerrado.")


2025-11-27 02:47:26 [INFO] convert_csv: [ConvertCSV] Iniciando job de conversão de csv para parquet.


2025-11-27 02:47:26 [INFO] file_io: [INFO] Encontrados 12 arquivo(s).


2025-11-27 02:47:26 [INFO] convert_csv: [ConvertCSV] Lendo arquivo csv: /opt/airflow/data-layer/raw/stage/airlines.csv.


2025-11-27 02:47:27 [INFO] quality_gates_raw: [Quality][Raw] Iniciando validações do dataset 'airlines.parquet'.


2025-11-27 02:47:28 [INFO] quality_gates_raw: [Quality][Raw]       _check_row_count_not_empty: airlines.parquet OK.


...

2025-11-27 02:47:29 [INFO] quality_gates_raw: [Quality][Raw]       _check_schema_columns: airports.parquet OK.


2025-11-27 02:47:29 [INFO] quality_gates_raw: [Quality][Raw] Todas as validações para 'airports.parquet' concluídas com sucesso.


2025-11-27 02:47:30 [INFO] convert_csv: [ConvertCSV] Arquivo convertido: /opt/airflow/data-layer/raw/stage/airports.parquet.


2025-11-27 02:47:30 [INFO] convert_csv: [ConvertCSV] Conversão concluída. 2 arquivo(s) processado(s).


2025-11-27 02:47:30 [INFO] convert_csv: [ConvertCSV] Job de conversão encerrado.


In [14]:
# Encerra a sessão Spark para o job
spark.stop()
log.info("[ConvertCSV] Sessão Spark finalizada.")


2025-11-27 02:47:30 [INFO] convert_csv: [ConvertCSV] Sessão Spark finalizada.


## Job 3: move_files_to_raw

Este job move os arquivos `parquet` da pasta **Stage** para a camada **Raw**, organizando-os por data de processamento.


In [15]:
log = get_logger("move_to_raw")

spark = get_spark_session("MoveStageToRaw")
log.info("[MoveToRaw] SparkSession iniciada.")


2025-11-27 02:47:31 [INFO] spark_helpers: [INFO] SparkSession criada: 'MoveStageToRaw' (master=local[*]).


2025-11-27 02:47:31 [INFO] move_to_raw: [MoveToRaw] SparkSession iniciada.


### Runner para o job `move_files_to_raw`

In [16]:
try:
    log.info("[MoveToRaw] Iniciando job de movimentação de arquivos.")

    parquet_files = check_files_in_folder(stage_path, "*.parquet")
    if not parquet_files:
        raise FileNotFoundError(f"[MoveToRaw][Error] Nenhum arquivo parquet encontrado em {stage_path}.")

    processing_date = datetime.now().strftime("%Y-%m-%d")

    move_files(
        spark=spark,
        source_files=parquet_files,
        base_dest_path=raw_path,
        processing_date=processing_date,
    )

    log.info(f"[MoveToRaw] {len(parquet_files)} arquivo(s) movido(s) para raw/{processing_date}.")

except Exception as e:
    log.exception(f"[MoveToRaw][Error] Falha durante execução: {e}.")
    raise

finally:
    log.info("[MoveToRaw] Job de movimentação encerrado.")


2025-11-27 02:47:31 [INFO] move_to_raw: [MoveToRaw] Iniciando job de movimentação de arquivos.


2025-11-27 02:47:31 [INFO] file_io: [INFO] Encontrados 3 arquivo(s).


2025-11-27 02:47:31 [INFO] file_io: [INFO] Movendo arquivos para '/opt/airflow/data-layer/raw'.


2025-11-27 02:47:31 [INFO] file_io: [INFO] 'airlines.parquet' movido para '/opt/airflow/data-layer/raw/2025-11-27/PARQUET/airlines.parquet'.


2025-11-27 02:47:32 [INFO] file_io: [INFO] 'airports.parquet' movido para '/opt/airflow/data-layer/raw/2025-11-27/PARQUET/airports.parquet'.


2025-11-27 02:47:36 [INFO] file_io: [INFO] 'flights.parquet' movido para '/opt/airflow/data-layer/raw/2025-11-27/PARQUET/flights.parquet'.


2025-11-27 02:47:36 [INFO] file_io: [INFO] Movimentação concluída com sucesso.


2025-11-27 02:47:36 [INFO] move_to_raw: [MoveToRaw] 3 arquivo(s) movido(s) para raw/2025-11-27.


2025-11-27 02:47:36 [INFO] move_to_raw: [MoveToRaw] Job de movimentação encerrado.


In [17]:
# Encerra a sessão Spark para o job
spark.stop()
log.info("[MoveToRaw] Sessão Spark finalizada.")


2025-11-27 02:47:36 [INFO] move_to_raw: [MoveToRaw] Sessão Spark finalizada.


## Job 4: load_raw_dbt

Este job carrega os **dados brutos** dos arquivos `airlines.parquet`, `airports.parquet` e `flights_part_1.csv` da pasta **Stage** para o schema `dbt_raw`, que servirá de base para as transformações realizadas pelo *dbt*. A opção por carregar apenas o arquivo `flights_part_1.csv` na tabela de voos foi tomada por motivos de performance.


In [18]:
log = get_logger("load_raw_dbt")
spark = get_spark_session("LoadRawdbt")

log.info("[LoadRaw][dbt] Sessão Spark iniciada.")


2025-11-27 02:47:36 [INFO] spark_helpers: [INFO] SparkSession criada: 'LoadRawdbt' (master=local[*]).


2025-11-27 02:47:36 [INFO] load_raw_dbt: [LoadRaw][dbt] Sessão Spark iniciada.


### Runner para o job `load_raw_dbt`

In [19]:
try:
    log.info("[LoadRaw][dbt] Iniciando job de carga dos dados para o dbt.")

    read_options = {
        "header": "true",
        "inferSchema": "false",
        "delimiter": ",",
    }

    # Carregar airlines.csv (RAW via Spark)
    try:
        airlines_path = Path(stage_path) / "airlines.csv"

        log.info(f"[dbt][Raw] Lendo airlines: {airlines_path}.")

        airlines_df = spark.read.options(**read_options).csv(str(airlines_path))

        log.info("[LoadRaw][dbt] Inserindo dados em 'dbt_raw.airlines'.")
        load_to_postgres(
            df=airlines_df,
            db_conn_id=postgres_conn_id,
            table_name="dbt_raw.airlines",
            mode="overwrite",
        )

    except Exception as e:
        log.exception(f"[LoadRaw][dbt][Error] Falha ao carregar airlines: {e}.")
        raise

    # Carregar airports.csv
    try:
        airports_path = Path(stage_path) / "airports.csv"

        log.info(f"[LoadRaw][dbt] Lendo airports: {airports_path}.")

        airports_df = spark.read.options(**read_options).csv(str(airports_path))

        log.info("[LoadRaw][dbt] Inserindo dados em 'dbt_raw.airports'.")
        load_to_postgres(
            df=airports_df,
            db_conn_id=postgres_conn_id,
            table_name="dbt_raw.airports",
            mode="overwrite",
        )

    except Exception as e:
        log.exception(f"[LoadRaw][dbt][Error] Falha ao carregar airports: {e}.")
        raise

    # Carregar flights_part_01.csv
    try:
        flights_chunk_path = Path(stage_path) / "flights_part_01.csv"

        log.info(f"[LoadRaw][dbt] Lendo chunk de flights: {flights_chunk_path}.")

        flights_df = spark.read.options(**read_options).csv(str(flights_chunk_path))

        log.info("[LoadRaw][dbt] Inserindo dados em 'dbt_raw.flights'.")
        load_to_postgres(
            df=flights_df,
            db_conn_id=postgres_conn_id,
            table_name="dbt_raw.flights",
            mode="overwrite",
        )

    except Exception as e:
        log.exception(f"[LoadRaw][dbt][Error] Falha ao carregar flights: {e}.")
        raise

    log.info("[LoadRaw][dbt] Carga concluída com sucesso.")

except Exception as e:
    log.exception(f"[LoadRaw][dbt][Error] Falha na execução do job: {e}.")
    raise

finally:
    log.info("[LoadRaw][dbt] Job de carga dos dados para o dbt encerrado.")


2025-11-27 02:47:36 [INFO] load_raw_dbt: [LoadRaw][dbt] Iniciando job de carga dos dados para o dbt.


2025-11-27 02:47:36 [INFO] load_raw_dbt: [dbt][Raw] Lendo airlines: /opt/airflow/data-layer/raw/stage/airlines.csv.


2025-11-27 02:47:37 [INFO] load_raw_dbt: [LoadRaw][dbt] Inserindo dados em 'dbt_raw.airlines'.


2025-11-27 02:47:37 [WARN] spark_helpers: [WARN] Airflow indisponível, usando variáveis de ambiente para conexão PostgreSQL.


2025-11-27 02:47:37 [INFO] spark_helpers: [LOAD] Limpando tabela 'dbt_raw.airlines' (TRUNCATE).


...

2025-11-27 02:47:39 [INFO] spark_helpers: [LOAD] Limpando tabela 'dbt_raw.flights' (TRUNCATE).


25/11/27 02:47:39 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


2025-11-27 02:47:50 [INFO] spark_helpers: [LOAD] Carga concluída em 'dbt_raw.flights' (modo=append).


2025-11-27 02:47:50 [INFO] load_raw_dbt: [LoadRaw][dbt] Carga concluída com sucesso.


2025-11-27 02:47:50 [INFO] load_raw_dbt: [LoadRaw][dbt] Job de carga dos dados para o dbt encerrado.


In [20]:
# Encerra a sessão Spark
spark.stop()
log.info("[LoadRaw][dbt] Sessão Spark finalizada.")


2025-11-27 02:47:50 [INFO] load_raw_dbt: [LoadRaw][dbt] Sessão Spark finalizada.


## Job 5: cleanup_stage

Este job remove os arquivos `csv` e `parquet` da pasta **Stage** após a conclusão da movimentação para a camada **Raw**.


In [21]:
log = get_logger("cleanup_stage")

spark = get_spark_session("CleanupStage")
log.info("[CleanupStage] SparkSession iniciada.")


2025-11-27 02:47:51 [INFO] spark_helpers: [INFO] SparkSession criada: 'CleanupStage' (master=local[*]).


2025-11-27 02:47:51 [INFO] cleanup_stage: [CleanupStage] SparkSession iniciada.


### Runner para o job `cleanup_stage`

In [22]:
try:
    log.info("[CleanupStage] Iniciando job de limpeza da stage.")

    removable_files = check_files_in_folder(stage_path, "*.*")
    if not removable_files:
        log.warning("[CleanupStage] Nenhum arquivo encontrado para remoção.")
    else:
        delete_files(spark, removable_files)
        log.info(f"[CleanupStage] {len(removable_files)} arquivo(s) removido(s) da stage.")

except Exception as e:
    log.exception(f"[CleanupStage][Error] Falha durante execução: {e}.")
    raise

finally:
    log.info("[CleanupStage] Job de limpeza encerrado.")


2025-11-27 02:47:51 [INFO] cleanup_stage: [CleanupStage] Iniciando job de limpeza da stage.


2025-11-27 02:47:51 [INFO] file_io: [INFO] Encontrados 12 arquivo(s).


2025-11-27 02:47:51 [INFO] file_io: [INFO] Deletando 12 arquivo(s).


2025-11-27 02:47:51 [INFO] file_io: [INFO] '/opt/airflow/data-layer/raw/stage/airlines.csv' deletado com sucesso.


2025-11-27 02:47:51 [INFO] file_io: [INFO] '/opt/airflow/data-layer/raw/stage/airports.csv' deletado com sucesso.


...

2025-11-27 02:47:51 [INFO] file_io: [INFO] '/opt/airflow/data-layer/raw/stage/flights_part_09.csv' deletado com sucesso.


2025-11-27 02:47:51 [INFO] file_io: [INFO] '/opt/airflow/data-layer/raw/stage/flights_part_10.csv' deletado com sucesso.


2025-11-27 02:47:51 [INFO] file_io: [INFO] Deleção concluída.


2025-11-27 02:47:51 [INFO] cleanup_stage: [CleanupStage] 12 arquivo(s) removido(s) da stage.


2025-11-27 02:47:51 [INFO] cleanup_stage: [CleanupStage] Job de limpeza encerrado.


In [23]:
# Encerra a sessão Spark para o job
spark.stop()
log.info("[CleanupStage] Sessão Spark finalizada.")


2025-11-27 02:47:52 [INFO] cleanup_stage: [CleanupStage] Sessão Spark finalizada.


# Silver

## Job 6: airlines_transform

Este job realiza a transformação e validação do dataset de companhias aéreas (`airlines.parquet`) da camada **Raw** para a camada **Silver**.

In [24]:
log = get_logger("transform_airlines")

spark = get_spark_session("TransformAirlines")
log.info("[Airlines] Sessão Spark iniciada.")


2025-11-27 02:47:53 [INFO] spark_helpers: [INFO] SparkSession criada: 'TransformAirlines' (master=local[*]).


2025-11-27 02:47:53 [INFO] transform_airlines: [Airlines] Sessão Spark iniciada.


### Definindo função de transformação do dataset `airlines`

In [25]:
def transform_airlines(df: DataFrame) -> DataFrame:
    """
    Transforma e valida o DataFrame de companhias aéreas para a camada silver.

    Args:
        df (DataFrame): DataFrame bruto da camada Bronze.

    Returns:
        DataFrame: DataFrame padronizado pronto para a camada Silver.
    """
    log.info("[Airlines] Iniciando transformações.")

    # Verifica se as colunas obrigatórias estão presentes
    log.info("[Airlines] Verificando presença de colunas obrigatórias.")
    required = {"IATA_CODE", "AIRLINE"}
    missing = required - set(df.columns)
    if missing:
        raise KeyError(f"[Airlines][ERROR] Colunas faltando no dataset: {missing}.")

    # Renomeia e converte tipos de colunas
    log.info("[Airlines] Renomeando e convertendo os tipos das colunas.")
    df2 = (
        df.withColumnRenamed("IATA_CODE", "airline_iata_code")
          .withColumnRenamed("AIRLINE", "airline_name")
          .withColumn("airline_iata_code", F.col("airline_iata_code").cast(StringType()))
          .withColumn("airline_name", F.col("airline_name").cast(StringType()))
    )

    # Padroniza os nomes das colunas para minúsculo
    log.info("[Airlines] Padronizando nomes de colunas para minúsculo.")
    df2 = df2.toDF(*[c.lower() for c in df2.columns])

    log.info("[Airlines] Transformação concluída com sucesso.")

    return df2


### Runner para o job `airlines_transform`

In [26]:
try:
    log.info("[Airlines] Iniciando job de trasnformação de 'airlines'.")

    # Resolve partição e caminhos
    source_partition = find_partition(raw_path, mode=run_mode, date_str=run_date)
    src = Path(raw_path) / source_partition / "PARQUET" / "airlines.parquet"
    dst = Path(silver_path) / source_partition / "PARQUET" / "airlines.parquet"

    if not src.exists():
        raise FileNotFoundError(f"[Airlines][ERROR] Arquivo não encontrado: {src}.")

    # Leitura do dataset bruto
    log.info(f"[Airlines] Lendo dataset: {src}.")
    df = spark.read.parquet(str(src))

    # Aplica transformações
    df_tf = transform_airlines(df)

    # Executa quality gates
    required_cols = ["airline_iata_code", "airline_name"]
    pk_cols = ["airline_iata_code"]
    
    log.info("[Airlines] Executando quality gates.")
    
    run_quality_gates_silver_base(
        df=df_tf,
        name="airlines_silver",
        required_columns=required_cols,
        pk_columns=pk_cols,
    )
    
    log.info("[Airlines] Quality gates concluídos com sucesso.")

    # Cria diretório de destino e grava o resultado na silver
    dst.parent.mkdir(parents=True, exist_ok=True)
    df_tf.write.mode("overwrite").parquet(str(dst))
    log.info(f"[Refinement][Airlines] Dataset salvo na camada silver: {dst}.")

except Exception as e:
    log.exception(f"[Airlines][ERROR] Falha na execução do job: {e}.")
    raise
finally:
    log.info("[Airlines] Job de trasnformação de 'airlines' encerrado.")


2025-11-27 02:47:53 [INFO] transform_airlines: [Airlines] Iniciando job de trasnformação de 'airlines'.


2025-11-27 02:47:53 [INFO] file_io: [INFO] Partição selecionada: 2025-11-27


2025-11-27 02:47:53 [INFO] transform_airlines: [Airlines] Lendo dataset: /opt/airflow/data-layer/raw/2025-11-27/PARQUET/airlines.parquet.


2025-11-27 02:47:54 [INFO] transform_airlines: [Airlines] Iniciando transformações.


2025-11-27 02:47:54 [INFO] transform_airlines: [Airlines] Verificando presença de colunas obrigatórias.


...

2025-11-27 02:48:01 [INFO] quality_gates_silver_base: [Quality][Silver]    _check_no_full_duplicates: airlines_silver OK.


2025-11-27 02:48:01 [INFO] quality_gates_silver_base: [Quality][Silver] Todas as validações para 'airlines_silver' concluídas com sucesso.


2025-11-27 02:48:01 [INFO] transform_airlines: [Airlines] Quality gates concluídos com sucesso.


2025-11-27 02:48:02 [INFO] transform_airlines: [Refinement][Airlines] Dataset salvo na camada silver: /opt/airflow/data-layer/silver/2025-11-27/PARQUET/airlines.parquet.


2025-11-27 02:48:02 [INFO] transform_airlines: [Airlines] Job de trasnformação de 'airlines' encerrado.


In [27]:
%%script false --no-raise-error # Comentar essa linha se estiver em debug ou se quiser rodar a célula.

df_tf.printSchema()

df_tf.limit(5).show(truncate=False)


In [28]:
# Encerra a sessão Spark
spark.stop()
log.info("[Airlines] Sessão Spark finalizada.")


2025-11-27 02:48:02 [INFO] transform_airlines: [Airlines] Sessão Spark finalizada.


## Job 7: airports_transform

Este job realiza a transformação e validação do dataset de aeroportos (`airports.parquet`) da camada **Raw** para a camada **Silver**.

In [29]:
log = get_logger("transform_airports")

spark = get_spark_session("TransformAirports")
log.info("[Airports] Sessão Spark iniciada.")


2025-11-27 02:48:03 [INFO] spark_helpers: [INFO] SparkSession criada: 'TransformAirports' (master=local[*]).


2025-11-27 02:48:03 [INFO] transform_airports: [Airports] Sessão Spark iniciada.


### Definindo função de trasformação do dataset `airports`

In [30]:
def transform_airports(df: DataFrame) -> DataFrame:
    """
    Transforma e valida o DataFrame de aeroportos para a camada silver.

    Args:
        df (DataFrame): DataFrame bruto lido da camada bronze.

    Returns:
        DataFrame: DataFrame limpo e padronizado para a camada silver.
    """
    log.info("[Airports] Iniciando transformações.")

    # Verifica colunas obrigatórias
    required = {"IATA_CODE", "LATITUDE", "LONGITUDE"}
    missing = required - set(df.columns)
    if missing:
        raise KeyError(f"[Airports][ERROR] Colunas faltando no dataset: {missing}.")

    # Correções de coordenadas faltosas
    corrections = {
        "ECP": {"LATITUDE": 30.3549, "LONGITUDE": -86.6160},
        "PBG": {"LATITUDE": 44.6895, "LONGITUDE": -68.0448},
        "UST": {"LATITUDE": 42.0703, "LONGITUDE": -87.9539},
    }

    # Tipagem e renomeação de colunas principais
    df2 = (
        df.withColumnRenamed("IATA_CODE", "airport_iata_code")
          .withColumn("airport_iata_code", F.col("airport_iata_code").cast(StringType()))
          .withColumn("LATITUDE", F.col("LATITUDE").cast(DoubleType()))
          .withColumn("LONGITUDE", F.col("LONGITUDE").cast(DoubleType()))
    )

    # Remove coluna COUNTRY
    if "COUNTRY" in df2.columns:
        df2 = df2.drop("COUNTRY")

    # Aplica correções manuais de coordenadas
    for code, coords in corrections.items():
        df2 = df2.withColumn(
            "LATITUDE",
            F.when(F.col("airport_iata_code") == code, F.lit(coords["LATITUDE"])).otherwise(F.col("LATITUDE"))
        ).withColumn(
            "LONGITUDE",
            F.when(F.col("airport_iata_code") == code, F.lit(coords["LONGITUDE"])).otherwise(F.col("LONGITUDE"))
        )

    # Renomeia colunas e força lowercase
    rename_map = {
        "AIRPORT": "airport_name",
        "CITY": "city",
        "STATE": "state",
        "LATITUDE": "latitude",
        "LONGITUDE": "longitude",
    }
    for old, new in rename_map.items():
        if old in df2.columns:
            df2 = df2.withColumnRenamed(old, new)

    # Normaliza nomes para minúsculo
    df2 = df2.toDF(*[c.lower() for c in df2.columns])

    log.info("[Airports] Transformação concluída com sucesso.")

    return df2


### Runner para o job `airports_transform`

In [31]:
try:
    log.info("[Airports] Iniciando job de trasnformação de 'airports'.")

    # Resolve partição e caminhos
    source_partition = find_partition(raw_path, mode=run_mode, date_str=run_date)
    src = Path(raw_path) / source_partition / "PARQUET" / "airports.parquet"
    dst = Path(silver_path) / source_partition / "PARQUET" / "airports.parquet"

    if not src.exists():
        raise FileNotFoundError(f"[Airports][ERROR] Arquivo não encontrado: {src}.")

    # Lê dataset e aplica transformações
    log.info(f"[Airports] Lendo dataset: {src}.")
    df = spark.read.parquet(str(src))
    
    df_tf = transform_airports(df)

    # Executa quality gates
    log.info("[Airports] Executando quality gates.")

    required_cols = ["airport_iata_code", "airport_name", "city", "state", "latitude", "longitude"]
    pk_cols = ["airport_iata_code"]

    run_quality_gates_silver_base(
        df=df_tf,
        name="airports_silver",
        required_columns=required_cols,
        pk_columns=pk_cols,
    )

    log.info("[Airports] Quality gates concluídos.")

    # Cria diretório e escreve resultado
    dst.parent.mkdir(parents=True, exist_ok=True)
    df_tf.write.mode("overwrite").parquet(str(dst))

    log.info(f"[Airports] Dataset salvo na camada silver: {dst}.")

except Exception as e:
    log.exception(f"[Airports][ERROR] Falha na execução do job: {e}")
    raise
finally:
    log.info("[Airports] Job de trasnformação de 'airports' encerrado.")


2025-11-27 02:48:03 [INFO] transform_airports: [Airports] Iniciando job de trasnformação de 'airports'.


2025-11-27 02:48:03 [INFO] file_io: [INFO] Partição selecionada: 2025-11-27


2025-11-27 02:48:03 [INFO] transform_airports: [Airports] Lendo dataset: /opt/airflow/data-layer/raw/2025-11-27/PARQUET/airports.parquet.


2025-11-27 02:48:03 [INFO] transform_airports: [Airports] Iniciando transformações.


2025-11-27 02:48:04 [INFO] transform_airports: [Airports] Transformação concluída com sucesso.


...

2025-11-27 02:48:07 [INFO] quality_gates_silver_base: [Quality][Silver]    _check_no_full_duplicates: airports_silver OK.


2025-11-27 02:48:07 [INFO] quality_gates_silver_base: [Quality][Silver] Todas as validações para 'airports_silver' concluídas com sucesso.


2025-11-27 02:48:07 [INFO] transform_airports: [Airports] Quality gates concluídos.


2025-11-27 02:48:08 [INFO] transform_airports: [Airports] Dataset salvo na camada silver: /opt/airflow/data-layer/silver/2025-11-27/PARQUET/airports.parquet.


2025-11-27 02:48:08 [INFO] transform_airports: [Airports] Job de trasnformação de 'airports' encerrado.


In [32]:
%%script false --no-raise-error # Comentar essa linha se estiver em debug ou se quiser rodar a célula.

df_tf.printSchema()

df_tf.limit(5).show(truncate=False)


In [33]:
# Encerra a sessão Spark
spark.stop()
log.info("[Airlines] Sessão Spark finalizada.")


2025-11-27 02:48:08 [INFO] transform_airports: [Airlines] Sessão Spark finalizada.


## Job 8: flights_transform

Este job transforma e normaliza o dataset de voos (`flights.parquet`) da **Raw** para a **Silver** (pré-join), gerando um arquivo intermediário a ser consumido na etapa de agregação.

In [34]:
log = get_logger("transform_flights_silver")

spark = get_spark_session("TransformFlightsSilver")
log.info("[Flights] Sessão Spark iniciada.")


2025-11-27 02:48:09 [INFO] spark_helpers: [INFO] SparkSession criada: 'TransformFlightsSilver' (master=local[*]).


2025-11-27 02:48:09 [INFO] transform_flights_silver: [Flights] Sessão Spark iniciada.


### Definindo funções de normalização das colunas de datas, de diferença de tempo e transformação

In [35]:
def normalize_time_expr(col_name: str) -> F.Column:
    """
    Normaliza valores de horário removendo casas decimais e preenchendo zeros à esquerda.
    """
    return F.when(
        F.col(col_name).isNotNull(),
        F.lpad(F.regexp_replace(F.col(col_name).cast("string"), r"\.0$", ""), 4, "0")
    )


def abs_min_diff(c1: str, c2: str) -> F.Column:
    """
    Retorna a diferença absoluta entre dois horários em minutos.
    """
    return F.abs(F.col(c1).cast("long") - F.col(c2).cast("long")) / 60.0


def transform_flights(df: DataFrame) -> DataFrame:
    """
    Transforma e valida o DataFrame de voos (pré-join) para a camada Silver.

    Args:
        df (DataFrame): DataFrame bruto lido da camada Bronze.

    Returns:
        DataFrame: DataFrame transformado e validado.
    """
    log.info("[Flights] Iniciando transformações.")

    # Validação mínima de colunas obrigatórias
    log.info("[Flights] Verificando presença de colunas obrigatórias.")
    required = {"YEAR", "MONTH", "DAY", "AIRLINE", "FLIGHT_NUMBER"}
    missing = required - set(df.columns)
    if missing:
        raise KeyError(f"[Flights][ERROR] Colunas faltando: {missing}.")

    # Filtro inicial e padronização de nomes
    log.info("[Flights] Aplicando filtros iniciais e padronizando nomes.")
    df2 = (
        df.filter((F.col("DIVERTED") != 1) & (F.col("CANCELLED") != 1))
          .toDF(*[c.lower() for c in df.columns])
          .withColumn("flight_date", to_date_from_ymd(F.col("year"), F.col("month"), F.col("day")))
          .withColumnRenamed("year", "flight_year")
          .withColumnRenamed("month", "flight_month")
          .withColumnRenamed("day", "flight_day")
          .withColumnRenamed("day_of_week", "flight_day_of_week")
    )

    # Normalização de horários
    log.info("[Flights] Normalização de colunas com horários.")
    time_cols = [
        "scheduled_departure", "departure_time",
        "scheduled_arrival", "arrival_time",
        "wheels_off", "wheels_on",
    ]

    for col_name in time_cols:
        if col_name in df2.columns:
            tmp = f"{col_name}_str"
            df2 = (
                df2.withColumn(tmp, normalize_time_expr(col_name))
                    .withColumn(
                        col_name,
                        F.to_timestamp(
                            F.concat_ws(" ", F.col("flight_date").cast("string"), F.col(tmp)),
                            "yyyy-MM-dd HHmm"
                        )
                    )
                    .drop(tmp)
            )

    # Detecção de horários trocados
    log.info("[Flights] Correção de horários trocados.")
    df2 = (
        df2
        .withColumn("diff_dep_sched_arr", abs_min_diff("departure_time", "scheduled_arrival"))
        .withColumn("diff_dep_sched_dep", abs_min_diff("departure_time", "scheduled_departure"))
        .withColumn("diff_arr_sched_dep", abs_min_diff("arrival_time", "scheduled_departure"))
        .withColumn("diff_arr_sched_arr", abs_min_diff("arrival_time", "scheduled_arrival"))
        .withColumn(
            "is_swapped",
            (F.col("diff_dep_sched_arr") < F.col("diff_dep_sched_dep")) &
            (F.col("diff_arr_sched_dep") < F.col("diff_arr_sched_arr"))
        )
        .withColumn(
            "departure_time_tmp",
            F.when(F.col("is_swapped"), F.col("arrival_time")).otherwise(F.col("departure_time"))
        )
        .withColumn(
            "arrival_time_tmp",
            F.when(F.col("is_swapped"), F.col("departure_time")).otherwise(F.col("arrival_time"))
        )
        .drop("departure_time", "arrival_time")
        .withColumnRenamed("departure_time_tmp", "departure_time")
        .withColumnRenamed("arrival_time_tmp", "arrival_time")
        .drop(
            "diff_dep_sched_arr", "diff_dep_sched_dep",
            "diff_arr_sched_dep", "diff_arr_sched_arr", "is_swapped"
        )
    )

    # Conversão numérica e substituição de nulos
    log.info("[Flights] Normalizando colunas numéricas.")
    numeric_cols = [
        "departure_delay", "arrival_delay", "taxi_out", "taxi_in",
        "air_time", "elapsed_time", "scheduled_time", "distance",
        "air_system_delay", "security_delay", "airline_delay",
        "late_aircraft_delay", "weather_delay",
    ]
    delay_cols = [
        "air_system_delay", "security_delay", "airline_delay",
        "late_aircraft_delay", "weather_delay",
    ]

    for c in numeric_cols:
        if c in df2.columns:
            expr = F.col(c).cast(DoubleType())
            if c in delay_cols:
                expr = F.coalesce(expr, F.lit(0.0))
            df2 = df2.withColumn(c, expr)

    # Ajuste de voos overnight
    log.info("[Flights] Ajustes nos voos que atravessa dia.")
    df2 = (
        df2.withColumn(
            "is_overnight_flight",
            F.when(
                (F.col("arrival_time").isNotNull()) &
                (F.col("departure_time").isNotNull()) &
                (F.hour(F.col("arrival_time")) < F.hour(F.col("departure_time"))),
                F.lit(True)
            ).otherwise(F.lit(False))
        )
        .withColumn(
            "arrival_time",
            F.when(F.col("is_overnight_flight"),
                   F.col("arrival_time") + F.expr("INTERVAL 1 DAY"))
             .otherwise(F.col("arrival_time"))
        )
    )

    # Filtros finais
    log.info("[Flights] Aplicando filtros finais.")
    df2 = df2.filter(
        (F.col("departure_time").isNotNull()) &
        (F.col("arrival_time").isNotNull()) &
        (F.col("arrival_time") > F.col("departure_time")) &
        (~F.col("origin_airport").rlike("^[0-9]+$")) &
        (~F.col("destination_airport").rlike("^[0-9]+$"))
    )

    # Remoção de colunas desnecessárias
    log.info("[Flights] Removendo colunas desnecessárias.")
    drop_cols = [c for c in ["diverted", "cancelled", "cancellation_reason"] if c in df2.columns]
    if drop_cols:
        df2 = df2.drop(*drop_cols)

    log.info("[Flights] Transformação concluída.")

    return df2


### Runner para o job `flights_transform`

In [36]:
try:
    log.info("[Flights] Iniciando job de trasnformação de 'flights'.")

    # Resolve partição e caminhos
    source_partition = find_partition(raw_path, mode=run_mode, date_str=run_date)
    src = Path(raw_path) / source_partition / "PARQUET" / "flights.parquet"
    dst_dir = Path(silver_path) / source_partition / "PARQUET"
    dst = dst_dir / "flights_pre_join.parquet"
    airports_src = dst_dir / "airports.parquet"

    if not src.exists():
        raise FileNotFoundError(f"[Flights][ERROR] Arquivo não encontrado: {src}.")
    if not airports_src.exists():
        raise FileNotFoundError(f"[Flights][ERROR] Airports não encontrado na silver: {airports_src}.")

    # Leitura e transformação
    log.info(f"[Flights] Lendo datasets: {src} e {airports_src}.")
    
    df = spark.read.parquet(str(src))
    airports_df = spark.read.parquet(str(airports_src))
    df_tf = transform_flights(df)
    
    # Quality gate
    run_quality_gates_silver_flights(df_tf, airports_df)

    # Escrita do resultado intermediário
    dst_dir.mkdir(parents=True, exist_ok=True)

    spark.conf.set("spark.sql.codegen.wholeStage", "false")
    df_tf.coalesce(1).write.mode("overwrite").parquet(str(dst))
    spark.conf.set("spark.sql.codegen.wholeStage", "true")

    log.info(f"[Flights] Dataset salvo na silver: {dst}.")

    # Libera cache
    df_tf.unpersist()

except Exception as e:
    log.exception(f"[Flights][ERROR] Falha na execução do job: {e}.")
    raise
finally:
    log.info("[Flights] Fim do job de transformação de 'flights'.")


2025-11-27 02:48:09 [INFO] transform_flights_silver: [Flights] Iniciando job de trasnformação de 'flights'.


2025-11-27 02:48:09 [INFO] file_io: [INFO] Partição selecionada: 2025-11-27


2025-11-27 02:48:09 [INFO] transform_flights_silver: [Flights] Lendo datasets: /opt/airflow/data-layer/raw/2025-11-27/PARQUET/flights.parquet e /opt/airflow/data-layer/silver/2025-11-27/PARQUET/airports.parquet.


2025-11-27 02:48:11 [INFO] transform_flights_silver: [Flights] Iniciando transformações.


2025-11-27 02:48:11 [INFO] transform_flights_silver: [Flights] Verificando presença de colunas obrigatórias.


...

2025-11-27 02:49:39 [INFO] quality_gates_silver_flights: [Quality][SilverFlights]     _check_delay_consistency: flights_silver OK.


2025-11-27 02:49:41 [INFO] quality_gates_silver_flights: [Quality][SilverFlights]     _check_referential_integrity: flights_silver OK.


2025-11-27 02:49:42 [INFO] quality_gates_silver_flights: [Quality][SilverFlights]     _check_referential_integrity: flights_silver OK.


2025-11-27 02:53:11 [INFO] transform_flights_silver: [Flights] Dataset salvo na silver: /opt/airflow/data-layer/silver/2025-11-27/PARQUET/flights_pre_join.parquet.


2025-11-27 02:53:11 [INFO] transform_flights_silver: [Flights] Fim do job de transformação de 'flights'.


In [37]:
%%script false --no-raise-error # Comentar essa linha se estiver em debug ou se quiser rodar a célula.

path_df = Path(silver_path) / "2025-11-12" / "PARQUET" / "flights_pre_join.parquet" # Verificar a data quando rodar

df = spark.read.parquet(str(path_df))

df.printSchema()

df.limit(5).show(truncate=True)


In [38]:
# Encerra a sessão Spark
spark.stop()
log.info("[Flights] Sessão Spark finalizada.")


2025-11-27 02:53:12 [INFO] transform_flights_silver: [Flights] Sessão Spark finalizada.


## Job 9: silver_aggregate

Este job realiza a agregação da camada **Silver**, unindo os datasets já tratados de `flights`, `airlines` e `airports` em um único dataset consolidado `flights_aggregated.parquet`, conforme o **DDL** da camada.

In [39]:
log = get_logger("silver_aggregate")

spark = get_spark_session("RefinementSilverAggregate")
log.info("[Aggregate] Sessão Spark iniciada.")


2025-11-27 02:53:13 [INFO] spark_helpers: [INFO] SparkSession criada: 'RefinementSilverAggregate' (master=local[*]).


2025-11-27 02:53:13 [INFO] silver_aggregate: [Aggregate] Sessão Spark iniciada.


### Definido função de agregação

In [40]:
def create_aggregated_flights_df(
    flights_silver_df: DataFrame,
    airlines_silver_df: DataFrame,
    airports_silver_df: DataFrame,
) -> DataFrame:
    """
    Constrói o DataFrame agregado da camada Silver (flights_aggregated), unindo:
        - flights_pre_join.parquet
        - airlines.parquet
        - airports.parquet

    Args:
        flights_silver_df (DataFrame): Dataset de voos já transformado (pré-join).
        airlines_silver_df (DataFrame): Dataset transformado de companhias aéreas.
        airports_silver_df (DataFrame): Dataset transformado de aeroportos.

    Returns:
        DataFrame: Dataset consolidado no formato final da camada Silver.
    """

    log.info("[Aggregate] Iniciando agregação dos datasets Silver.")

    # Detecta coluna de companhia aérea no dataset 'flights'
    if "airline_iata_code" in flights_silver_df.columns:
        airline_col = "airline_iata_code"
    elif "airline" in flights_silver_df.columns:
        airline_col = "airline"
    else:
        raise KeyError("Nenhuma coluna de companhia aérea encontrada no dataset flights.")

    # Detecta colunas de origem e destino no dataset 'flights'
    if "origin_airport_iata_code" in flights_silver_df.columns:
        origin_col = "origin_airport_iata_code"
    elif "origin_airport" in flights_silver_df.columns:
        origin_col = "origin_airport"
    else:
        raise KeyError("Coluna de aeroporto de origem não encontrada.")

    if "dest_airport_iata_code" in flights_silver_df.columns:
        dest_col = "dest_airport_iata_code"
    elif "destination_airport" in flights_silver_df.columns:
        dest_col = "destination_airport"
    else:
        raise KeyError("Coluna de aeroporto de destino não encontrada.")

    # Join com airlines
    df_joined = flights_silver_df.join(
        airlines_silver_df,
        flights_silver_df[airline_col] == airlines_silver_df["airline_iata_code"],
        how="left",
    )

    # Seleciona campos para aeroportos de origem
    df_origin = (
        airports_silver_df.select(
            F.col("airport_iata_code").alias("origin_airport_iata_code"),
            F.col("airport_name").alias("origin_airport_name"),
            F.col("city").alias("origin_city"),
            F.col("state").alias("origin_state"),
            F.col("latitude").alias("origin_latitude"),
            F.col("longitude").alias("origin_longitude"),
        )
    )

    # Seleciona campos para aeroportos de destino
    df_dest = (
        airports_silver_df.select(
            F.col("airport_iata_code").alias("dest_airport_iata_code"),
            F.col("airport_name").alias("dest_airport_name"),
            F.col("city").alias("dest_city"),
            F.col("state").alias("dest_state"),
            F.col("latitude").alias("dest_latitude"),
            F.col("longitude").alias("dest_longitude"),
        )
    )

    # Join com aeroportos de origem e destino
    df_joined = (
        df_joined.join(
            df_origin,
            df_joined[origin_col] == F.col("origin_airport_iata_code"),
            how="left",
        )
        .join(
            df_dest,
            df_joined[dest_col] == F.col("dest_airport_iata_code"),
            how="left",
        )
    )

    # Mapeamento dos tipos
    schema_casts = {
        "flight_id": "bigint",
        "flight_year": "smallint",
        "flight_month": "smallint",
        "flight_day": "smallint",
        "flight_day_of_week": "smallint",
        "flight_date": "date",

        "airline_iata_code": "string",
        "airline_name": "string",

        "flight_number": "int",
        "tail_number": "string",

        "origin_airport_iata_code": "string",
        "origin_airport_name": "string",
        "origin_city": "string",
        "origin_state": "string",
        "origin_latitude": "double",
        "origin_longitude": "double",

        "dest_airport_iata_code": "string",
        "dest_airport_name": "string",
        "dest_city": "string",
        "dest_state": "string",
        "dest_latitude": "double",
        "dest_longitude": "double",

        "scheduled_departure": "timestamp",
        "departure_time": "timestamp",
        "scheduled_arrival": "timestamp",
        "arrival_time": "timestamp",
        "wheels_off": "timestamp",
        "wheels_on": "timestamp",

        "departure_delay": "double",
        "arrival_delay": "double",
        "taxi_out": "double",
        "taxi_in": "double",
        "air_time": "double",
        "elapsed_time": "double",
        "scheduled_time": "double",
        "distance": "double",

        "is_overnight_flight": "boolean",

        "air_system_delay": "double",
        "security_delay": "double",
        "airline_delay": "double",
        "late_aircraft_delay": "double",
        "weather_delay": "double",
    }

    for col_name, spark_type in schema_casts.items():
        if col_name in df_joined.columns:
            df_joined = df_joined.withColumn(col_name, F.col(col_name).cast(spark_type))

    # Seleção final conforme o ddl
    final_df = df_joined.select(
        "flight_year",
        "flight_month",
        "flight_day",
        "flight_day_of_week",
        "flight_date",
        "airline_iata_code",
        "airline_name",
        "flight_number",
        "tail_number",
        "origin_airport_iata_code",
        "origin_airport_name",
        "origin_city",
        "origin_state",
        "origin_latitude",
        "origin_longitude",
        "dest_airport_iata_code",
        "dest_airport_name",
        "dest_city",
        "dest_state",
        "dest_latitude",
        "dest_longitude",
        "scheduled_departure",
        "departure_time",
        "scheduled_arrival",
        "arrival_time",
        "wheels_off",
        "wheels_on",
        "departure_delay",
        "arrival_delay",
        "taxi_out",
        "taxi_in",
        "air_time",
        "elapsed_time",
        "scheduled_time",
        "distance",
        "air_system_delay",
        "security_delay",
        "airline_delay",
        "late_aircraft_delay",
        "weather_delay",
    )

    # Cria pk sequencial ordenada
    window_spec = (
        Window
        .orderBy(
            F.col("flight_date").asc(),
            F.col("airline_iata_code").asc(),
            F.col("flight_number").asc(),
            F.col("origin_airport_iata_code").asc(),
            F.col("departure_time").asc(),
        )
    )

    # Gera pk determinística e sequencial
    final_df = final_df.withColumn(
        "flight_id",
        F.row_number().over(window_spec)
    )

    # Reordena colunas para a pk ser a primeira
    final_df = final_df.select(
        "flight_id",
        *[c for c in final_df.columns if c != "flight_id"]
    )

    log.info("[Aggregate] Agregação concluída com sucesso.")

    return final_df


### Runner para o job `silver_aggregate`

In [41]:
try:
    log.info("[Aggregate] Iniciando job de agregação da camada Silver.")

    # Resolve partição e caminhos
    source_partition: str = find_partition(
        base_path=silver_path,
        mode=run_mode,
        date_str=run_date,
    )
    base_dir: Path = Path(silver_path) / source_partition / "PARQUET"

    flights_path: Path  = base_dir / "flights_pre_join.parquet"
    airlines_path: Path = base_dir / "airlines.parquet"
    airports_path: Path = base_dir / "airports.parquet"

    # Verifica existência dos arquivos necessários
    for required_file in [flights_path, airlines_path, airports_path]:
        if not required_file.exists():
            raise FileNotFoundError(
                f"[Refinement][Aggregate][ERROR] Arquivo esperado não encontrado: {required_file}"
            )

    # Leitura dos datasets
    log.info("[Aggregate] Lendo datasets silver (flights, airlines, airports).")

    df_flights  = spark.read.parquet(str(flights_path))
    df_airlines = spark.read.parquet(str(airlines_path))
    df_airports = spark.read.parquet(str(airports_path))

    # Construção do DataFrame agregado
    aggregated_df: DataFrame = create_aggregated_flights_df(
        flights_silver_df=df_flights,
        airlines_silver_df=df_airlines,
        airports_silver_df=df_airports,
    )

    # Filtra registros inválidos de aeroportos
    aggregated_df = aggregated_df.filter(
        F.col("origin_airport_iata_code").isNotNull()
        & F.col("dest_airport_iata_code").isNotNull()
    )

    # Quality gates
    run_quality_gates_silver_aggregated(aggregated_df)

    # Escrita do arquivo final (para debug)
    output_path: Path = base_dir / "flights_aggregated.parquet"
    aggregated_df.coalesce(1).write.mode("overwrite").parquet(str(output_path))

    log.info(f"[Aggregate] Dataset agregado salvo em: {output_path}")

    # Libera cache após uso
    aggregated_df.unpersist()

except Exception as e:
    log.exception(f"[Aggregate][ERROR] Falha na execução do job: {e}.")
    raise

finally:
    log.info("[Aggregate] Job de agregação encerrado.")


2025-11-27 02:53:13 [INFO] silver_aggregate: [Aggregate] Iniciando job de agregação da camada Silver.


2025-11-27 02:53:13 [INFO] file_io: [INFO] Partição selecionada: 2025-11-27


2025-11-27 02:53:13 [INFO] silver_aggregate: [Aggregate] Lendo datasets silver (flights, airlines, airports).


2025-11-27 02:53:13 [INFO] silver_aggregate: [Aggregate] Iniciando agregação dos datasets Silver.


2025-11-27 02:53:15 [INFO] silver_aggregate: [Aggregate] Agregação concluída com sucesso.


...

25/11/27 02:53:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/27 02:53:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


25/11/27 02:53:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/27 02:53:46 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


25/11/27 02:54:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
25/11/27 02:54:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


2025-11-27 02:55:43 [INFO] silver_aggregate: [Aggregate] Dataset agregado salvo em: /opt/airflow/data-layer/silver/2025-11-27/PARQUET/flights_aggregated.parquet


2025-11-27 02:55:43 [INFO] silver_aggregate: [Aggregate] Job de agregação encerrado.


In [42]:
%%script false --no-raise-error # Comentar essa linha se estiver em debug ou se quiser rodar a célula.

aggregated_df.printSchema()

aggregated_df.limit(5).show(truncate=True)


In [43]:
# Encerra a sessão Spark
spark.stop()
log.info("[Aggregate] Sessão Spark finalizada.")


2025-11-27 02:55:44 [INFO] silver_aggregate: [Aggregate] Sessão Spark finalizada.


## Job 10: silver_load

Este job realiza a carga do dataset agregado da camada **Silver** (`flights_aggregated.parquet`) para o PostgreSQL, populando a tabela `silver.silver_flights` conforme o ddl da camada.

In [44]:
log = get_logger("silver_load")

spark = get_spark_session("SilverLoad")
log.info("[SilverLoad] Sessão Spark iniciada.")


2025-11-27 02:55:44 [INFO] spark_helpers: [INFO] SparkSession criada: 'SilverLoad' (master=local[*]).


2025-11-27 02:55:44 [INFO] silver_load: [SilverLoad] Sessão Spark iniciada.


### Runner para o job `silver_load`

In [45]:
try:
    log.info("[SilverLoad] Iniciando execução do job de carga.")

    # Resolve partição e caminhos
    partition = find_partition(
        base_path=silver_path,
        mode=run_mode,
        date_str=run_date,
    )

    base_dir = Path(silver_path) / partition / "PARQUET"
    parquet_path = base_dir / "flights_aggregated.parquet"

    if not parquet_path.exists():
        raise FileNotFoundError(
            f"[SilverLoad][ERROR] Arquivo não encontrado: {parquet_path}."
        )

    log.info(f"[SilverLoad] Lendo dataset agregado: {parquet_path}.")
    df = spark.read.parquet(str(parquet_path))

    log.info(f"[SilverLoad] Tuplas encontradas: {df.count():,}.")

    # Carga para PostgreSQL
    try:
        log.info("[SilverLoad] Inserindo dados em silver.silver_flights.")

        load_to_postgres(
            df=df,
            db_conn_id=postgres_conn_id,
            table_name="silver.silver_flights",
        )

        log.info("[SilverLoad] Validando tuplas da tabela após a carga.")
        
        # Valida carga
        expected_count = df.count()
        assert_table_rowcount(
            db_conn_id=postgres_conn_id,
            table_name="silver.silver_flights",
            expected_count=expected_count,
        )

        log.info("[SilverLoad] Validação pós-carga encerrada.")

    except Exception as e:
        log.exception(f"[Refinement][SilverLoad][ERROR] Erro durante carga no PostgreSQL: {e}")
        raise

    log.info("[SilverLoad] Carga concluída com sucesso.")

except Exception as e:
    log.exception(f"[SilverLoad][ERROR] Falha na execução: {e}")
    raise
finally:
    log.info("[SilverLoad] Job de carga encerrado.")


2025-11-27 02:55:44 [INFO] silver_load: [SilverLoad] Iniciando execução do job de carga.


2025-11-27 02:55:44 [INFO] file_io: [INFO] Partição selecionada: 2025-11-27


2025-11-27 02:55:44 [INFO] silver_load: [SilverLoad] Lendo dataset agregado: /opt/airflow/data-layer/silver/2025-11-27/PARQUET/flights_aggregated.parquet.


2025-11-27 02:55:45 [INFO] silver_load: [SilverLoad] Tuplas encontradas: 5,208,259.


2025-11-27 02:55:45 [INFO] silver_load: [SilverLoad] Inserindo dados em silver.silver_flights.


...

2025-11-27 03:05:12 [INFO] postgres_helpers: [AssertRowCount] Esperado: 5,208,259 | Encontrado: 5,208,259


2025-11-27 03:05:12 [INFO] postgres_helpers: [AssertRowCount] Validação concluída com sucesso.


2025-11-27 03:05:12 [INFO] silver_load: [SilverLoad] Validação pós-carga encerrada.


2025-11-27 03:05:12 [INFO] silver_load: [SilverLoad] Carga concluída com sucesso.


2025-11-27 03:05:12 [INFO] silver_load: [SilverLoad] Job de carga encerrado.


In [46]:
%%script false --no-raise-error # Comentar essa linha se estiver em debug ou se quiser rodar a célula.

df.printSchema()

df.limit(2).show(truncate=False)


In [47]:
# Encerra a sessão Spark
spark.stop()
log.info("[SilverLoad] Sessão Spark finalizada.")


2025-11-27 03:05:12 [INFO] silver_load: [SilverLoad] Sessão Spark finalizada.


## Jog 11: silver_cleanup

Este job remove arquivos intermediários da camada **Silver**, mantendo apenas o arquivo final `flights_aggregated.parquet` na partição.

In [48]:
log = get_logger("silver_cleanup")

spark = get_spark_session("SilverCleanupJob")
log.info("[SilverCleanup] Sessão Spark iniciada.")


2025-11-27 03:05:13 [INFO] spark_helpers: [INFO] SparkSession criada: 'SilverCleanupJob' (master=local[*]).


2025-11-27 03:05:13 [INFO] silver_cleanup: [SilverCleanup] Sessão Spark iniciada.


### Runner para o job `silver_cleanup`

In [49]:
try:
    log.info("[SilverCleanup] Iniciando job de limpeza da camada silver.")

    # Resolve partição e caminhos
    partition = find_partition(
        base_path=silver_path,
        mode=run_mode,
        date_str=run_date,
    )
    partition_dir = Path(silver_path) / partition / "PARQUET"

    if not partition_dir.exists():
        raise FileNotFoundError(
            f"[Refinement][SilverCleanup][ERROR] Diretório de partição não encontrado: {partition_dir}."
        )

    # Apenas este diretório deve permanecer (debug)
    keep = {"flights_aggregated.parquet"}

    log.info(f"[SilverCleanup] Diretório alvo: {partition_dir}.")
    log.info(f"[SilverCleanup] Mantendo apenas: {keep}.")

    for item in partition_dir.iterdir():
        # Se for o agregado, não mexe
        if item.name in keep:
            log.info(f"[SilverCleanup] Mantendo: {item}")
            continue

        # Remove diretórios intermediários
        if item.is_dir():
            try:
                shutil.rmtree(item)
                log.info(f"[SilverCleanup] Diretório removido: {item}.")
            except Exception as e:
                log.warning(f"[SilverCleanup] Falha ao remover diretório {item}: {e}.")
            continue

        # Remove arquivos soltos
        if item.is_file():
            try:
                item.unlink()
                log.info(f"[SilverCleanup] Arquivo removido: {item}.")
            except Exception as e:
                log.warning(f"[SilverCleanup] Falha ao remover arquivo {item}: {e}.")

    log.info("[SilverCleanup] Limpeza concluída com sucesso.")

except Exception as e:
    log.exception(f"[SilverCleanup][ERROR] Falha durante o cleanup: {e}")
    raise

finally:
    log.info("[SilverCleanup] Job de limpeza da camada silver encerrado.")


2025-11-27 03:05:13 [INFO] silver_cleanup: [SilverCleanup] Iniciando job de limpeza da camada silver.


2025-11-27 03:05:13 [INFO] file_io: [INFO] Partição selecionada: 2025-11-27


2025-11-27 03:05:13 [INFO] silver_cleanup: [SilverCleanup] Diretório alvo: /opt/airflow/data-layer/silver/2025-11-27/PARQUET.


2025-11-27 03:05:13 [INFO] silver_cleanup: [SilverCleanup] Mantendo apenas: {'flights_aggregated.parquet'}.


2025-11-27 03:05:13 [INFO] silver_cleanup: [SilverCleanup] Diretório removido: /opt/airflow/data-layer/silver/2025-11-27/PARQUET/airlines.parquet.


2025-11-27 03:05:13 [INFO] silver_cleanup: [SilverCleanup] Diretório removido: /opt/airflow/data-layer/silver/2025-11-27/PARQUET/airports.parquet.


2025-11-27 03:05:13 [INFO] silver_cleanup: [SilverCleanup] Mantendo: /opt/airflow/data-layer/silver/2025-11-27/PARQUET/flights_aggregated.parquet


2025-11-27 03:05:14 [INFO] silver_cleanup: [SilverCleanup] Diretório removido: /opt/airflow/data-layer/silver/2025-11-27/PARQUET/flights_pre_join.parquet.


2025-11-27 03:05:14 [INFO] silver_cleanup: [SilverCleanup] Limpeza concluída com sucesso.


2025-11-27 03:05:14 [INFO] silver_cleanup: [SilverCleanup] Job de limpeza da camada silver encerrado.


In [50]:
%%script false --no-raise-error # Comentar essa linha se estiver em debug ou se quiser rodar a célula.

partition = find_partition(silver_path, mode=run_mode, date_str=run_date)
for item in (Path(silver_path) / partition / "PARQUET").iterdir():
    print(item)


In [51]:
# Encerra a sessão Spark
spark.stop()
log.info("[SilverCleanup] Sessão Spark finalizada.")


2025-11-27 03:05:14 [INFO] silver_cleanup: [SilverCleanup] Sessão Spark finalizada.
